# Scraping UFC Stats

This is to clarify what is necessary for scraping and upkeep of database

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
from random import randint
import  random
import os
os.chdir('C:/Users/Travis/OneDrive/Data Science/Personal_Projects/Sports/UFC_Prediction_V2')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
import pickle
from sklearn.metrics import fbeta_score
import winsound
from bs4 import BeautifulSoup
import time


### Initial Scraping Functions

In [2]:
def get_event_date_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    date = soup.find('li', class_='b-list__box-list-item').text
    date = date.replace('\n', '')
    ed = date.find(':')
    date = date[ed+2:].strip()
    return date

In [3]:
get_event_date_from_ufcstats('http://www.ufcstats.com/event-details/93bf96be327fcd98')

'September 10, 2022'

In [4]:
def get_details_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    details = soup.find('div', class_='b-fight-details__content').text
    details = details.replace('\n', '')
    deet = details.find('Details:')
    details = details[deet+8:].strip()
    return details

In [5]:
get_details_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'Guillotine Choke From Bottom Guard'

In [6]:
def get_event_title_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_='b-content__title').text
    title = title.replace('\n', '').strip()
    return title

In [7]:
get_event_title_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'UFC 279: Diaz vs. Ferguson'

In [8]:
def get_event_url_from_ufcstats(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('h2', class_='b-content__title')
    title = title.find('a')
    title = title['href']
    return title

In [9]:
get_event_url_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'http://www.ufcstats.com/event-details/93bf96be327fcd98'

In [10]:
def get_fight_urls(urls):

    # get links to all fights on each event page

    links = pd.DataFrame()
    for u in urls:
        try:
            reqs = requests.get(u)
            soup = BeautifulSoup(reqs.text, 'html.parser')
            title = soup.find('h2', class_='b-content__title').text
            title = title.replace('\n', '')
            date = soup.find('li', class_='b-list__box-list-item').text
            d = date.find('Date:')
            date = date[d+6:].replace('\n', '')
            for link in soup.find_all('a'):
                # append url and event title to df
                if 'fight-details' in link.get('href'):
                    links = links.append({'Fight_url': link.get('href'), 'Event_title': title, 'Date' : date}, ignore_index=True)

        except:
            return 'No Fighties'
            
    return links

In [11]:
# test get_fight_urls
get_fight_urls(['http://www.ufcstats.com/event-details/93bf96be327fcd98'])

,Fight_url,Event_title,Date
0,http://www.ufcstats.com/fight-details/7e8fd03e...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
1,http://www.ufcstats.com/fight-details/0111e6a6...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
2,http://www.ufcstats.com/fight-details/d3e23d7d...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
3,http://www.ufcstats.com/fight-details/989b8d36...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
4,http://www.ufcstats.com/fight-details/d869eaea...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
5,http://www.ufcstats.com/fight-details/0e6d6240...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
6,http://www.ufcstats.com/fight-details/301f230c...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
7,http://www.ufcstats.com/fight-details/77acb967...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
8,http://www.ufcstats.com/fight-details/bf334cb4...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"
9,http://www.ufcstats.com/fight-details/db3e26d0...,UFC 279: Diaz vs. Ferguson ...,"September 10, 2022"


In [12]:
def get_winner_from_ufcstats(url):

    ### This function scrapes the UFCStats website for the winner of a given fight###

    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        both = soup.find_all('div', class_='b-fight-details__person')
        first = both[0].text.replace('   ', '').replace('\n', '').strip()
        second = both[1].text.replace('   ', '').replace('\n', '').strip()
        if(first.startswith('W')):
            winner = first[3:]
        else:
            winner = second[3:]
        return winner
        
    except:
        return nan

In [13]:
# test
get_winner_from_ufcstats('http://www.ufcstats.com/fight-details/7e8fd03e070d0c25')

'Nate Diaz'

In [14]:
def get_event_date_from_ufcstats(url):
    ### This function scrapes the UFCStats website for the event date of a given fight ###
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        date = soup.find('li', class_='b-list__box-list-item').text
        date = date.replace('\n', '')
        ed = date.find(':')
        date = date[ed+2:].strip()
        return date
    except:
        return nan

In [15]:
def get_event_details(event_id):
    # This function scrapes event details from ufc stats

    #try:
        df = pd.read_html('http://www.ufcstats.com/event-details/' + event_id)
        df = df[0]
        # split by space and get third element
        df['Fighter1'] = df['Fighter'].str.split('  ').str[0]
        df['Fighter2'] = df['Fighter'].str.split('  ').str[1]
        # fix Kd
        df['F1_Kd'] = df['Kd'].str.split('  ').str[0]
        df['F2_Kd'] = df['Kd'].str.split('  ').str[1]
        # fix Str
        df['F1_Str'] = df['Str'].str.split('  ').str[0]
        df['F2_Str'] = df['Str'].str.split('  ').str[1]
        # fix Td
        df['F1_Td'] = df['Td'].str.split('  ').str[0]
        df['F2_Td'] = df['Td'].str.split('  ').str[1]
        # fix Sub
        df['F1_Sub'] = df['Sub'].str.split('  ').str[0]
        df['F2_Sub'] = df['Sub'].str.split('  ').str[1]

        badcols = ['Fighter', 'Kd', 'Str', 'Td', 'Sub']

        # save index+1 to new column called "fight_num"
        df['fight_num'] = df.index + 1
        df['event_id'] = event_id

        # get fight links
        page = requests.get('http://www.ufcstats.com/event-details/' + event_id)
        soup = BeautifulSoup(page.content, 'html.parser')
        fight_linkies = soup.find_all('a')
        fight_links = [n['href'] for n in fight_linkies]
        fight_links = [n for n in fight_links if 'fight-details' in n]
        # delete duplicate links
        fight_links2 = pd.DataFrame(fight_links)
        fight_links2 = fight_links2.drop_duplicates()
        fight_links = fight_links2[0].tolist()

        # append fight links to df
        df['fight_link'] = fight_links

        df = df.drop(badcols, axis=1)
        df.to_csv('data/ufc_stats/events2/'+ event_id +'.csv')
        return df

In [16]:
# test 
test_1 = get_event_details('a23e63184c65f5b8').head(3)
test_1

,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,F2_Kd,F1_Str,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link
0,win,Bantamweight,KO/TKO Kick,4,2:17,Marlon Vera,Dominick Cruz,3,0,61,92,0,2,0,0,1,a23e63184c65f5b8,http://www.ufcstats.com/fight-details/11cc331f...
1,win,Featherweight,M-DEC,3,5:00,Nate Landwehr,David Onama,0,1,91,71,3,1,2,0,2,a23e63184c65f5b8,http://www.ufcstats.com/fight-details/efd7efba...
2,win,Women's Strawweight,U-DEC,3,5:00,Yazmin Jauregui,Iasmin Lucindo,0,0,86,66,0,0,0,0,3,a23e63184c65f5b8,http://www.ufcstats.com/fight-details/833d67f4...


In [17]:
def get_fight_totals(fight_details_url):
    # function returns the fight totals for a given fightid
    da_url = fight_details_url
    fightid = da_url[da_url.find('fight-details/')+14:]
    df = pd.read_html(da_url)
    # Part 1: Total Fight Stats
    totals = df[0]
    # replace '---' with 0
    for col in totals.columns:
        totals[col] = totals[col].astype(str).replace({'---': 0})
        totals[col] = totals[col].astype(str).replace({'--': 0})


    totals['Fighter_A'] = totals['Fighter'].str.split(' ').str[0] + ' ' + totals['Fighter'].str.split(' ').str[1]
    totals['Fighter_B'] = totals['Fighter'].str.split(' ').str[2] + ' ' + totals['Fighter'].str.split(' ').str[3]
    totals['A_Kd'] = totals['KD'].str.split('  ').str[0]
    totals['B_Kd'] = totals['KD'].str.split('  ').str[1]
    totals['A_Sig_strike'] = totals['Sig. str.'].str.split('  ').str[0]
    totals['B_Sig_strike'] = totals['Sig. str.'].str.split('  ').str[1]
    # split sig strikes into landed and attempted
    totals['A_Sig_strike_land'] = totals['A_Sig_strike'].str.split(' of ').str[0]
    totals['A_Sig_strike_att'] = totals['A_Sig_strike'].str.split(' of ').str[1]
    totals['B_Sig_strike_land'] = totals['B_Sig_strike'].str.split(' of ').str[0]
    totals['B_Sig_strike_att'] = totals['B_Sig_strike'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Sig_strike_land'].replace({'---', 0}, inplace=True)
    totals['A_Sig_strike_att'].replace({'---', 0}, inplace=True)
    totals['B_Sig_strike_land'].replace({'---', 0}, inplace=True)
    totals['B_Sig_strike_att'].replace({'---', 0}, inplace=True)

    totals['A_Sig_strike_land'] = pd.to_numeric(totals['A_Sig_strike_land'])
    totals['A_Sig_strike_att'] = pd.to_numeric(totals['A_Sig_strike_att'])
    totals['B_Sig_strike_land'] = pd.to_numeric(totals['B_Sig_strike_land'])
    totals['B_Sig_strike_att'] = pd.to_numeric(totals['B_Sig_strike_att'])

    totals['A_Sig_strike_percent'] = totals['Sig. str. %'].str.split('  ').str[0]
    totals['B_Sig_strike_percent'] = totals['Sig. str. %'].str.split('  ').str[1]
    # get rid of % sign
    totals['A_Sig_strike_percent'] = totals['A_Sig_strike_percent'].str.replace('%', '')
    totals['B_Sig_strike_percent'] = totals['B_Sig_strike_percent'].str.replace('%', '')

    totals['A_Sig_strike_percent'] = totals['A_Sig_strike_percent'].astype(str).replace({'---': 0})
    totals['B_Sig_strike_percent'] = totals['B_Sig_strike_percent'].astype(str).replace({'---': 0})
    # change to numeric
    totals['A_Sig_strike_percent'] = pd.to_numeric(totals['A_Sig_strike_percent'])/100
    totals['B_Sig_strike_percent'] = pd.to_numeric(totals['B_Sig_strike_percent'])/100
    # total strikes
    totals['A_Total_Strikes'] = totals['Total str.'].str.split('  ').str[0]
    totals['B_Total_Strikes'] = totals['Total str.'].str.split('  ').str[1]
    # split total strikes into landed and attempted
    totals['A_Total_Strikes_land'] = totals['A_Total_Strikes'].str.split(' of ').str[0]
    totals['A_Total_Strikes_att'] = totals['A_Total_Strikes'].str.split(' of ').str[1]
    totals['B_Total_Strikes_land'] = totals['B_Total_Strikes'].str.split(' of ').str[0]
    totals['B_Total_Strikes_att'] = totals['B_Total_Strikes'].str.split(' of ').str[1]

    totals['A_Total_Strikes_land'].replace({'---',0}, inplace=True)
    totals['A_Total_Strikes_att'].replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_land'].replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_att'].replace({'---',0}, inplace=True)
    # change to numeric
    totals['A_Total_Strikes_land'] = pd.to_numeric(totals['A_Total_Strikes_land'])
    totals['A_Total_Strikes_att'] = pd.to_numeric(totals['A_Total_Strikes_att'])
    totals['B_Total_Strikes_land'] = pd.to_numeric(totals['B_Total_Strikes_land'])
    totals['B_Total_Strikes_att'] = pd.to_numeric(totals['B_Total_Strikes_att'])

    totals['A_Total_Strikes_land'].astype(str).replace({'---',0}, inplace=True)
    totals['A_Total_Strikes_att'].astype(str).replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_land'].astype(str).replace({'---',0}, inplace=True)
    totals['B_Total_Strikes_att'].astype(str).replace({'---',0}, inplace=True) 

    # calculate total strike percentage
    totals['A_Total_Strikes_percent'] = totals['A_Total_Strikes_land'] / totals['A_Total_Strikes_att']
    totals['B_Total_Strikes_percent'] = totals['B_Total_Strikes_land'] / totals['B_Total_Strikes_att']
    # takedown stats
    totals['A_Takedowns'] = totals['Td'].str.split('  ').str[0]
    totals['B_Takedowns'] = totals['Td'].str.split('  ').str[1]
    # split takedowns into landed and attempted
    totals['A_Takedowns_land'] = totals['A_Takedowns'].str.split(' of ').str[0]
    totals['A_Takedowns_att'] = totals['A_Takedowns'].str.split(' of ').str[1]
    totals['B_Takedowns_land'] = totals['B_Takedowns'].str.split(' of ').str[0]
    totals['B_Takedowns_att'] = totals['B_Takedowns'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Takedowns_land'] = pd.to_numeric(totals['A_Takedowns_land'])
    totals['A_Takedowns_att'] = pd.to_numeric(totals['A_Takedowns_att'])
    totals['B_Takedowns_land'] = pd.to_numeric(totals['B_Takedowns_land'])
    totals['B_Takedowns_att'] = pd.to_numeric(totals['B_Takedowns_att'])
    # fix %
    totals['Td %'].replace({'---': 0}, inplace=True)
    # get rid of ---
    totals['A_Takedowns_land'].replace({'---': 0}, inplace=True)
    totals['A_Takedowns_att'].replace({'---': 0}, inplace=True)
    totals['B_Takedowns_land'].replace({'---': 0}, inplace=True)
    totals['B_Takedowns_att'].replace({'---': 0}, inplace=True)
    totals['A_Takedown_percent'] = totals['A_Takedowns_land'] / totals['A_Takedowns_att']
    totals['B_Takedown_percent'] = totals['B_Takedowns_land'] / totals['B_Takedowns_att']
    # submission attempts
    totals['A_Sub_Attempts'] = totals['Sub. att'].str.split('  ').str[0]
    totals['B_Sub_Attempts'] = totals['Sub. att'].str.split('  ').str[1]
    # split submission attempts into landed and attempted
    totals['A_Sub_Attempts_land'] = totals['A_Sub_Attempts'].str.split(' of ').str[0]
    totals['A_Sub_Attempts_att'] = totals['A_Sub_Attempts'].str.split(' of ').str[1]
    totals['B_Sub_Attempts_land'] = totals['B_Sub_Attempts'].str.split(' of ').str[0]
    totals['B_Sub_Attempts_att'] = totals['B_Sub_Attempts'].str.split(' of ').str[1]
    # change to numeric
    totals['A_Sub_Attempts_land'] = pd.to_numeric(totals['A_Sub_Attempts_land'])
    totals['A_Sub_Attempts_att'] = pd.to_numeric(totals['A_Sub_Attempts_att'])
    totals['B_Sub_Attempts_land'] = pd.to_numeric(totals['B_Sub_Attempts_land'])
    totals['B_Sub_Attempts_att'] = pd.to_numeric(totals['B_Sub_Attempts_att'])
    # Rev
    totals['A_Rev'] = totals['Rev.'].str.split('  ').str[0]
    totals['B_Rev'] = totals['Rev.'].str.split('  ').str[1]
    # change to numeric
    totals['A_Rev'] = pd.to_numeric(totals['A_Rev'])
    totals['B_Rev'] = pd.to_numeric(totals['B_Rev'])
    # control time
    totals['A_Ctrl_time'] = totals['Ctrl'].str.split('  ').str[0]
    totals['B_Ctrl_time'] = totals['Ctrl'].str.split('  ').str[1]
    

    # split control time into minutes and seconds, and then convert to seconds
    totals['A_Ctrl_time_min'] = totals['A_Ctrl_time'].str.split(':').str[0]
    totals['A_Ctrl_time_sec'] = totals['A_Ctrl_time'].str.split(':').str[1]
    totals['B_Ctrl_time_min'] = totals['B_Ctrl_time'].str.split(':').str[0]
    totals['B_Ctrl_time_sec'] = totals['B_Ctrl_time'].str.split(':').str[1]

    totals['A_Ctrl_time_min'] = totals['A_Ctrl_time_min'].replace({'--': 0})
    totals['A_Ctrl_time_sec'] = totals['A_Ctrl_time_sec'].replace({'--': 0})
    totals['B_Ctrl_time_min'] = totals['B_Ctrl_time_min'].replace({'--': 0})
    totals['B_Ctrl_time_sec'] = totals['B_Ctrl_time_sec'].replace({'--': 0})

    # convert to seconds
    totals['A_Ctrl_time_tot'] = pd.to_numeric(totals['A_Ctrl_time_min']) * 60 + pd.to_numeric(totals['A_Ctrl_time_sec'])
    totals['B_Ctrl_time_tot'] = pd.to_numeric(totals['B_Ctrl_time_min']) * 60 + pd.to_numeric(totals['B_Ctrl_time_sec'])


    # add date with beatufil soup
    totals['details'] = get_details_from_ufcstats(da_url)
    totals['event_title'] = get_event_title_from_ufcstats(da_url)
    event_url = get_event_url_from_ufcstats(da_url)
    totals['event_url'] = event_url
    totals['date'] = get_event_date_from_ufcstats(event_url)

    try:
        totals['Winner']= get_winner_from_ufcstats(da_url)
    except:
        totals['Winner']= nan

    bad_cols = ['Fighter', 'KD', 'Sig. str. %', 'Sig. str.', 'Total str.', 'Td', 'Td %', 'Sub. att', 
                'Rev.', 'Ctrl', 'A_Sig_strike', 'B_Sig_strike', 'A_Takedowns', 'B_Takedowns',
                'A_Sub_Attempts', 'B_Sub_Attempts', 'A_Ctrl_time', 'B_Ctrl_time']
    totals.drop(columns=bad_cols, inplace=True)
    totals.to_csv('data/ufc_stats/fight_totals2/' + fightid + '_totals.csv')
    return totals
    


In [18]:
# test
get_fight_totals('http://www.ufcstats.com/fight-details/a419e6df765aa755')

,Fighter_A,Fighter_B,A_Kd,B_Kd,A_Sig_strike_land,A_Sig_strike_att,B_Sig_strike_land,B_Sig_strike_att,A_Sig_strike_percent,B_Sig_strike_percent,...,A_Ctrl_time_sec,B_Ctrl_time_min,B_Ctrl_time_sec,A_Ctrl_time_tot,B_Ctrl_time_tot,details,event_title,event_url,date,Winner
0,Stephen Thompson,Kevin Holland,0,0,163,260,119,210,0.62,0.56,...,47,1,21,47,81,to Corner Stoppage,UFC Fight Night: Thompson vs. Holland,http://www.ufcstats.com/event-details/b23388ff...,"December 03, 2022","Stephen Thompson ""Wonderboy"""


In [19]:
def get_significant_strikes(url):
    df = pd.read_html(url)
    # Part 2: Significant Strikes
    ss = df[2]
     
    ss['Fighter_A'] = ss['Fighter'].str.split(' ').str[0] + ' ' + ss['Fighter'].str.split(' ').str[1]
    ss['Fighter_B'] = ss['Fighter'].str.split(' ').str[2] + ' ' + ss['Fighter'].str.split(' ').str[3]
    ss['A_Head_Strikes'] = ss['Head'].str.split('  ').str[0]
    ss['B_Head_Strikes'] = ss['Head'].str.split('  ').str[1]
    # split head strikes into landed and attempted
    ss['A_Head_Strikes_land'] = ss['A_Head_Strikes'].str.split(' of ').str[0]
    ss['A_Head_Strikes_att'] = ss['A_Head_Strikes'].str.split(' of ').str[1]
    ss['B_Head_Strikes_land'] = ss['B_Head_Strikes'].str.split(' of ').str[0]
    ss['B_Head_Strikes_att'] = ss['B_Head_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Head_Strikes_land'] = pd.to_numeric(ss['A_Head_Strikes_land'])
    ss['A_Head_Strikes_att'] = pd.to_numeric(ss['A_Head_Strikes_att'])
    ss['B_Head_Strikes_land'] = pd.to_numeric(ss['B_Head_Strikes_land'])
    ss['B_Head_Strikes_att'] = pd.to_numeric(ss['B_Head_Strikes_att'])
    # Head Strikes Percentage
    ss['A_Head_Strikes_percent'] = ss['A_Head_Strikes_land'] / ss['A_Head_Strikes_att']
    ss['B_Head_Strikes_percent'] = ss['B_Head_Strikes_land'] / ss['B_Head_Strikes_att']
    # Body Strikes
    ss['A_Body_Strikes'] = ss['Body'].str.split('  ').str[0]
    ss['B_Body_Strikes'] = ss['Body'].str.split('  ').str[1]
    # split body strikes into landed and attempted
    ss['A_Body_Strikes_land'] = ss['A_Body_Strikes'].str.split(' of ').str[0]
    ss['A_Body_Strikes_att'] = ss['A_Body_Strikes'].str.split(' of ').str[1]
    ss['B_Body_Strikes_land'] = ss['B_Body_Strikes'].str.split(' of ').str[0]
    ss['B_Body_Strikes_att'] = ss['B_Body_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Body_Strikes_land'] = pd.to_numeric(ss['A_Body_Strikes_land'])
    ss['A_Body_Strikes_att'] = pd.to_numeric(ss['A_Body_Strikes_att'])
    ss['B_Body_Strikes_land'] = pd.to_numeric(ss['B_Body_Strikes_land'])
    ss['B_Body_Strikes_att'] = pd.to_numeric(ss['B_Body_Strikes_att'])
    # Body Strikes Percentage
    ss['A_Body_Strikes_percent'] = ss['A_Body_Strikes_land'] / ss['A_Body_Strikes_att']
    ss['B_Body_Strikes_percent'] = ss['B_Body_Strikes_land'] / ss['B_Body_Strikes_att']
    # Leg Strikes
    ss['A_Leg_Strikes'] = ss['Leg'].str.split('  ').str[0]
    ss['B_Leg_Strikes'] = ss['Leg'].str.split('  ').str[1]
    # split leg strikes into landed and attempted
    ss['A_Leg_Strikes_land'] = ss['A_Leg_Strikes'].str.split(' of ').str[0]
    ss['A_Leg_Strikes_att'] = ss['A_Leg_Strikes'].str.split(' of ').str[1]
    ss['B_Leg_Strikes_land'] = ss['B_Leg_Strikes'].str.split(' of ').str[0]
    ss['B_Leg_Strikes_att'] = ss['B_Leg_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Leg_Strikes_land'] = pd.to_numeric(ss['A_Leg_Strikes_land'])
    ss['A_Leg_Strikes_att'] = pd.to_numeric(ss['A_Leg_Strikes_att'])
    ss['B_Leg_Strikes_land'] = pd.to_numeric(ss['B_Leg_Strikes_land'])
    ss['B_Leg_Strikes_att'] = pd.to_numeric(ss['B_Leg_Strikes_att'])
    # Leg Strikes Percentage
    ss['A_Leg_Strikes_percent'] = ss['A_Leg_Strikes_land'] / ss['A_Leg_Strikes_att']
    ss['B_Leg_Strikes_percent'] = ss['B_Leg_Strikes_land'] / ss['B_Leg_Strikes_att']
    # Distance Strikes
    ss['A_Distance_Strikes'] = ss['Distance'].str.split('  ').str[0]
    ss['B_Distance_Strikes'] = ss['Distance'].str.split('  ').str[1]
    # split distance strikes into landed and attempted
    ss['A_Distance_Strikes_land'] = ss['A_Distance_Strikes'].str.split(' of ').str[0]
    ss['A_Distance_Strikes_att'] = ss['A_Distance_Strikes'].str.split(' of ').str[1]
    ss['B_Distance_Strikes_land'] = ss['B_Distance_Strikes'].str.split(' of ').str[0]
    ss['B_Distance_Strikes_att'] = ss['B_Distance_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Distance_Strikes_land'] = pd.to_numeric(ss['A_Distance_Strikes_land'])
    ss['A_Distance_Strikes_att'] = pd.to_numeric(ss['A_Distance_Strikes_att'])
    ss['B_Distance_Strikes_land'] = pd.to_numeric(ss['B_Distance_Strikes_land'])
    ss['B_Distance_Strikes_att'] = pd.to_numeric(ss['B_Distance_Strikes_att'])
    # distance strikes percentage
    ss['A_Distance_Strikes_percent'] = ss['A_Distance_Strikes_land'] / ss['A_Distance_Strikes_att']
    ss['B_Distance_Strikes_percent'] = ss['B_Distance_Strikes_land'] / ss['B_Distance_Strikes_att']

    # Clinch Strikes
    ss['A_Clintch_Strikes'] = ss['Clinch'].str.split('  ').str[0]
    ss['B_Clintch_Strikes'] = ss['Clinch'].str.split('  ').str[1]
    # split clinch strikes into landed and attempted
    ss['A_Clinch_Strikes_land'] = ss['A_Clintch_Strikes'].str.split(' of ').str[0]
    ss['A_Clinch_Strikes_att'] = ss['A_Clintch_Strikes'].str.split(' of ').str[1]
    ss['B_Clinch_Strikes_land'] = ss['B_Clintch_Strikes'].str.split(' of ').str[0]
    ss['B_Clinch_Strikes_att'] = ss['B_Clintch_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Clinch_Strikes_land'] = pd.to_numeric(ss['A_Clinch_Strikes_land'])
    ss['A_Clinch_Strikes_att'] = pd.to_numeric(ss['A_Clinch_Strikes_att'])
    ss['B_Clinch_Strikes_land'] = pd.to_numeric(ss['B_Clinch_Strikes_land'])
    ss['B_Clinch_Strikes_att'] = pd.to_numeric(ss['B_Clinch_Strikes_att'])
    # clinch strikes percentage
    ss['A_Clinch_Strikes_percent'] = ss['A_Clinch_Strikes_land'] / ss['A_Clinch_Strikes_att']
    ss['B_Clinch_Strikes_percent'] = ss['B_Clinch_Strikes_land'] / ss['B_Clinch_Strikes_att']

    # Ground Strikes
    ss['A_Ground_Strikes'] = ss['Ground'].str.split('  ').str[0]
    ss['B_Ground_Strikes'] = ss['Ground'].str.split('  ').str[1]
    # split ground strikes into landed and attempted
    ss['A_Ground_Strikes_land'] = ss['A_Ground_Strikes'].str.split(' of ').str[0]
    ss['A_Ground_Strikes_att'] = ss['A_Ground_Strikes'].str.split(' of ').str[1]
    ss['B_Ground_Strikes_land'] = ss['B_Ground_Strikes'].str.split(' of ').str[0]
    ss['B_Ground_Strikes_att'] = ss['B_Ground_Strikes'].str.split(' of ').str[1]
    # change to numeric
    ss['A_Ground_Strikes_land'] = pd.to_numeric(ss['A_Ground_Strikes_land'])
    ss['A_Ground_Strikes_att'] = pd.to_numeric(ss['A_Ground_Strikes_att'])
    ss['B_Ground_Strikes_land'] = pd.to_numeric(ss['B_Ground_Strikes_land'])
    ss['B_Ground_Strikes_att'] = pd.to_numeric(ss['B_Ground_Strikes_att'])
    # ground strikes percentage
    ss['A_Ground_Strikes_percent'] = ss['A_Ground_Strikes_land'] / ss['A_Ground_Strikes_att']
    ss['B_Ground_Strikes_percent'] = ss['B_Ground_Strikes_land'] / ss['B_Ground_Strikes_att']
    # add stuff with beautiful soup
    ss['details'] = get_details_from_ufcstats(url)
    ss['event_title'] = get_event_title_from_ufcstats(url)
    event_url = get_event_url_from_ufcstats(url)
    ss['event_url'] = event_url

    fightid = url[url.find('fight-details/')+14:]

    bad_cols = ['Fighter', 'Sig. str', 'Sig. str. %', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground',
                'A_Head_Strikes', 'B_Head_Strikes', 'A_Body_Strikes', 'B_Body_Strikes', 'A_Leg_Strikes', 'B_Leg_Strikes',
                'A_Distance_Strikes', 'B_Distance_Strikes', 'A_Clintch_Strikes', 'B_Clintch_Strikes', 'A_Ground_Strikes', 
                'B_Ground_Strikes']
    ss.drop(columns=bad_cols, inplace=True)
    ss.to_csv('data/ufc_stats/sig_strikes2/' + fightid + '_sigstrikes.csv')

    return ss

In [20]:
# get all events from oddsbyfighter
all_event_odds = pd.read_csv('data/final/odds/All_Odds_by_Fighter_WithChange.csv')
all_event_odds

,fighter,5D,Ref,event_odds_url,event_ufcstats_url,event_id,event_name,odds_change
0,Brock Lesnar,-225.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-31.0
1,Frank Mir,205.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,59.0
2,Georges St-Pierre,-275.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,NaN
3,Thiago Alves,250.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,-6.0
4,Dan Henderson,-200.0,NaN,https://www.bestfightodds.com/events/ufc-100-137,http://ufcstats.com/event-details/0ee783aa00e4...,ufc-100-137,UFC 100,4.0
...,...,...,...,...,...,...,...,...
24391,Krzysztof Jotko,NaN,-124.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-32.0
24392,Chelsea Chandler,NaN,113.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-6.0
24393,Julija Stoliarenko,NaN,-133.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,4.0
24394,Guido Cannetti,NaN,250.0,https://www.bestfightodds.com/events/ufc-vegas...,http://ufcstats.com/event-details/4f670b7972fa...,ufc-vegas-57-2489,UFC Fight Night: Dern vs. Yan,-22.0


In [21]:
# get all events from all_event_odds

all_event_odds_urls = all_event_odds['event_ufcstats_url'].unique()
print(len(all_event_odds_urls))
all_event_odds_urls

500


array(['http://ufcstats.com/event-details/0ee783aa00e468f0',
       'http://ufcstats.com/event-details/48d1f690b763934c',
       'http://ufcstats.com/event-details/c6a33ff198aaaeeb',
       'http://ufcstats.com/event-details/896c322f56b8be5a',
       'http://ufcstats.com/event-details/fa8b9e6b0c2269f8',
       'http://ufcstats.com/event-details/84a067c46306a737',
       'http://ufcstats.com/event-details/821cd80aab70d5f9',
       'http://ufcstats.com/event-details/a6d8bfe9e0c8153b',
       'http://ufcstats.com/event-details/15edcf67ccf5be84',
       'http://ufcstats.com/event-details/49efbdc6c9f650c4',
       'http://ufcstats.com/event-details/c80095f6092271a7',
       'http://ufcstats.com/event-details/24b033b3daf1c9df',
       'http://ufcstats.com/event-details/d0c29452d3272603',
       'http://ufcstats.com/event-details/cfb65863d5099327',
       'http://ufcstats.com/event-details/140745cbbcb023ac',
       'http://ufcstats.com/event-details/054defd5420a551f',
       'http://ufcstats.

In [22]:
all_event_urls = all_event_odds_urls.tolist()

In [23]:
# get all the fight links from an event page, dl to events folder

#TODO: Do we need to do this for all events, or just NEW events?

events_with_fight_links = pd.DataFrame()
m = 1
errors = []

for event_url in all_event_urls:
    try:
        event_id = event_url.split('/')[-1]
        get_event_details(event_id)
        print(f'{m} of {len(all_event_urls)}')
        m += 1
    except:
        errors.append(event_url)
        print(f'ERROR: {m} of {len(all_event_urls)}')
        m += 1

print(len(errors))
errors


1 of 500
2 of 500
3 of 500
4 of 500
5 of 500
6 of 500
7 of 500
8 of 500
9 of 500
10 of 500
11 of 500
12 of 500
13 of 500
14 of 500
15 of 500
16 of 500
17 of 500
18 of 500
19 of 500
20 of 500
21 of 500
22 of 500
23 of 500
24 of 500
25 of 500
26 of 500
27 of 500
28 of 500
29 of 500
30 of 500
31 of 500
32 of 500
33 of 500
34 of 500
35 of 500
36 of 500
37 of 500
38 of 500
39 of 500
40 of 500
41 of 500
42 of 500
43 of 500
44 of 500
45 of 500
46 of 500
47 of 500
48 of 500
49 of 500
50 of 500
51 of 500
52 of 500
53 of 500
54 of 500
55 of 500
56 of 500
57 of 500
58 of 500
59 of 500
60 of 500
61 of 500
62 of 500
63 of 500
64 of 500
65 of 500
66 of 500
67 of 500
68 of 500
69 of 500
70 of 500
71 of 500
72 of 500
73 of 500
74 of 500
75 of 500
76 of 500
77 of 500
78 of 500
79 of 500
80 of 500
81 of 500
82 of 500
83 of 500
84 of 500
85 of 500
86 of 500
87 of 500
88 of 500
89 of 500
90 of 500
91 of 500
92 of 500
93 of 500
94 of 500
95 of 500
96 of 500
97 of 500
98 of 500
99 of 500
100 of 500
101 of 5

[]

In [24]:
# Aggregate all the fight data from all the events
all_event_data = pd.DataFrame()
m = 1
errors = []

event_files = os.listdir('data/ufc_stats/events2/')

for event_file in event_files:
    data = pd.read_csv('data/ufc_stats/events2/' + event_file)
    all_event_data = all_event_data.append(data, ignore_index=True)

all_event_data

,Unnamed: 0,W/L,Weight class,Method,Round,Time,Fighter1,Fighter2,F1_Kd,F2_Kd,F1_Str,F2_Str,F1_Td,F2_Td,F1_Sub,F2_Sub,fight_num,event_id,fight_link
0,0,win,Heavyweight,KO/TKO Punches,3,4:23,Ciryl Gane,Tai Tuivasa,1,1,110,29,0,0,0,0,1,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/810c9da0...
1,1,win,Middleweight,U-DEC,3,5:00,Robert Whittaker,Marvin Vettori,0,0,74,33,1,0,0,0,2,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/b8ca1acd...
2,2,win,Middleweight,U-DEC,3,5:00,Nassourdine Imavov,Joaquin Buckley,0,0,51,46,2,0,2,0,3,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/20f316f9...
3,3,win,Middleweight,KO/TKO Punches,3,1:09,Roman Kopylov,Alessio Di Chirico,1,0,59,39,0,0,0,0,4,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/1a191251...
4,4,win,Featherweight,U-DEC,3,5:00,William Gomis,Jarno Errens,0,0,32,20,3,0,0,1,5,00a905a4a4a2b071,http://www.ufcstats.com/fight-details/398db400...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5770,7,win,Featherweight,SUB Guillotine Choke,3,2:59,Brian Ortega,Renato Moicano,0,0,65,109,0,2,1,0,8,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/4924d7a9...
5771,8,win,Featherweight,U-DEC,3,5:00,Calvin Kattar,Andre Fili,0,0,75,73,2,0,0,0,9,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/3d95b4f9...
5772,9,win,Women's Strawweight,U-DEC,3,5:00,Aleksandra Albu,Kailin Curran,0,0,82,86,5,1,0,0,10,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/e80a8338...
5773,10,win,Flyweight,S-DEC,3,5:00,Jarred Brooks,Eric Shelton,0,0,27,31,5,1,1,1,11,ff9578cdbfabd323,http://www.ufcstats.com/fight-details/4f480695...


In [25]:
all_event_data.to_csv('data/final/events/All_Events_Fights_and_FightUrls.csv')